In [ ]:
import numpy as np
import pandas as pd
import h5py
import os
from tqdm.notebook import tqdm

from methods.io_things import tar_to_downscaled_models
from methods.plotting import plotly_plot
from methods.base import BaseConfig

# CURR_DIR = os.getcwd()

ROCKS = pd.read_excel('rock_properties.xlsx', index_col=0)
ROCKS

In [ ]:
params = BaseConfig(shape = (21, 21, 21)) # !!! <- careful

PATH_TO_MODELS = 'D:/geological_models/' # path to folder with TAR files like "DYKE_FOLD_FOLT" and others
# PATH_TO_MODELS = 'del_me/'

NMODELS = 30 # of each type. Total number of models woul be = 343 * NMODELS. Put it over 9000 to process all models

yield_path = tar_to_downscaled_models(PATH_TO_MODELS, NMODELS, target_shape=params.shape, fname='downscaled_models')

In [ ]:
yield_path = 'downscaled_models_03_05_2024__13_33_23.h5'

with h5py.File(yield_path, 'r') as f:
    perm = f['perm'][2]
    # dens = f['dens'][2]

plotly_plot(perm)
